In [ ]:
import numpy as np
import pdb

import meerkat as mk
from meerkat.contrib.eeg import build_stanford_eeg_dp
from domino.utils import split_dp

## Load EEG datapanel

In [ ]:
dp_art = build_stanford_eeg_dp.out(run_id=409)
dp=dp_art.load()
dp.lz[0]

In [ ]:
len(dp)

In [ ]:
len(dp.lz[~dp["id"].duplicated()])

## split data

In [ ]:
dp_splits_art = split_dp.out(412) #split_dp(dp_art, split_on="patient_id")
dp_splits = dp_splits_art.load()
print(len(dp_splits))
dp_splits.head()

In [ ]:
dp_splits['id'].nunique()

In [ ]:
age_labels = dp["age"] > 10
sz_labels = dp["target"]
np.corrcoef([age_labels,sz_labels])[1,0]

## Slice dp based on metadata

In [ ]:
from domino.slices.eeg import build_slice

#dp_train = dp.lz[dp["split"]=="train"]
dp_age_art = build_slice("correlation", 409, dp_splits_art, correlate="age", corr=0.9, n=2500, correlate_threshold=10)
dp_age = dp_age_art.load()

In [ ]:
len(dp_age)

In [ ]:
age_labels = dp_age["age"] > 10
sz_labels = dp_age["target"]
np.corrcoef([age_labels,sz_labels])[0,1]

In [ ]:
(dp_age["split"]=="valid").sum()

## Get multiple slices

In [ ]:
from domino.slices.eeg import collect_correlation_slices

# correlate_list = ["age"]
# corr_list = [0, 0.3, 0.5, 0.9]
# correlate_thresholds = [10]
# dp_run_id = 409
# dp_slices_art = collect_correlation_slices(dp_run_id, correlate_list, corr_list, correlate_thresholds)
dp_slices_art = collect_correlation_slices.out(439)
dp_slices = dp_slices_art.load()
dp_slices.head()